In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data extraction

In [95]:
import pandas as pd

def read_data(file_path, sheet_name):
    """
    read excel data

    parameter:
    file_path: Excel file path。
    sheet_name: worksheet name。

    return:
    df: read DataFrame。
    """
    return pd.read_excel(file_path, sheet_name=sheet_name)

def split_and_adjust_data(df):
    """
    Sperate dataframe as two tables, and change columns name

    parameter:
    df: original DataFrame。

    Return:
    df1, df2: sperate and adjust as DataFrame。
    """
    # find out all missing value
    full_missing_rows = df[df.isna().all(axis=1)]

    if full_missing_rows.empty:
        # If there are no rows with missing values, return the original data and an empty DataFrame
        return df, pd.DataFrame()

    # The next row index of the last row with all missing values
    last_missing_index = full_missing_rows.index[-1]

    # Separate into two tables: df1 is the part before the missing values, df2 is the part after the missing values
    df1 = df.iloc[:last_missing_index]
    df2 = df.iloc[last_missing_index + 1:]

    # df1 uses the third row as the table header
    df1.columns = df1.iloc[2]
    df1 = df1[3:].reset_index(drop=True)  # Data starting from the fouth row and resetting the index


    # The third row of df2 is used as the header
    df2.columns = df2.iloc[2]  # The third line is used as header
    df2 = df2[3:].reset_index(drop=True)  # Data starting from the fourth row and resetting the index

    # Make sure the first column of the table header of df1 and df2 is 'Base Run' and the second column is 'Date'
    df1.columns = ['Base Run' if i == 0 else df1.columns[i] for i in range(len(df1.columns))]
    df1.columns = ['Base Run', 'Date'] + df1.columns[2:].tolist()

    df2.columns = ['Base Run' if i == 0 else df2.columns[i] for i in range(len(df2.columns))]
    df2.columns = ['Base Run', 'Date'] + df2.columns[2:].tolist()

    # Remove blank values ​​from the column names of df2 and move the column name of the fourth column forward to the third column
    new_columns = [col if pd.notna(col) else None for col in df2.columns]
    new_columns = [col for col in new_columns if col is not None]

    # If the third column is blank, replace it with the name of the fourth column and add an Unknown column name
    if new_columns[2] is None:
        new_columns[2] = new_columns[3]
        new_columns[3] = 'Unknown'  # replace with Unknown
    else:
        new_columns.append('Unknown')  # If the third column is not blank, add an Unknown column name at the end

    # Make sure the number of column names matches the number of columns of df2
    if len(new_columns) < len(df2.columns):
        new_columns.append('Unknown')

    df2.columns = new_columns

    return df1, df2

# File path and worksheet name
file_path = '/content/drive/MyDrive/713 Capstone Project (DA)/RUN3PM10QUS_base.xlsx'
sheet_name = 'Contributions'

# Reading Excel Files
df = read_data(file_path, sheet_name)

# Separate data into two tables and adjust column names
df1, df2 = split_and_adjust_data(df)



# Drop the rightmost column of df1 and df2
df1 = df1.drop(df1.columns[-1], axis=1)
df2 = df2.drop(df2.columns[-1], axis=1)  # 移除补充的Unknown列

# print results
print("First Table (df1):")
print(df1)
print("\nSecond Table (df2):")
print(df2)

First Table (df1):
     Base Run       Date Soil/Road dust Sea salt Diesel vehicles  \
0           1 2006-01-06        0.52359    1.237          1.6154   
1           1 2006-01-07        0.54113    0.894          1.0639   
2           1 2006-01-08        0.81198   1.4498          1.1626   
3           1 2006-01-09         1.0009    1.917         0.81576   
4           1 2006-01-11        0.69889   1.7211         0.75511   
...       ...        ...            ...      ...             ...   
3671        1 2022-12-03        0.37984   1.1617         0.25032   
3672        1 2022-12-09       -0.16833  0.55372         0.20551   
3673        1 2022-12-15        0.20323  0.15687         0.12945   
3674        1 2022-12-21        0.54835   0.3482         0.22288   
3675        1 2022-12-27        0.40598  0.64801        -0.10268   

     Biomass burning Construction Petrol vehicles Sulphate/Marine diesel  
0            0.65492       0.0653         0.91454                0.59039  
1            0

# Create Database

In [96]:
import sqlite3
import os

def drop_database(db_filename):
    # If the database file exists, delete it
    if os.path.exists(db_filename):
        os.remove(db_filename)
        print(f"Database file {db_filename} has been deleted.")
    else:
        print(f"Database file {db_filename} does not exist.")

def create_database(db_filename):
    # Connect to a SQLite database, this will create a new empty database file
    conn = sqlite3.connect(db_filename)

    # SQL statement to create a table
    create_dates_table = """
    CREATE TABLE IF NOT EXISTS Dates (
        date_id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT NOT NULL
    );
    """
    create_factors_table = """
    CREATE TABLE IF NOT EXISTS Factors (
        factor_id INTEGER PRIMARY KEY AUTOINCREMENT,
        factor_name TEXT NOT NULL
    );
    """
    create_contributions_table = """
    CREATE TABLE IF NOT EXISTS Contributions (
        contribution_id INTEGER PRIMARY KEY AUTOINCREMENT,
        date_id INTEGER,
        factor_id INTEGER,
        contribution_value REAL,
        FOREIGN KEY (date_id) REFERENCES Dates(date_id),
        FOREIGN KEY (factor_id) REFERENCES Factors(factor_id)
    );
    """

    # Execute SQL statements to create tables
    cursor = conn.cursor()
    cursor.execute(create_dates_table)
    cursor.execute(create_factors_table)
    cursor.execute(create_contributions_table)

    # confirm operation
    conn.commit()

    # close connections
    conn.close()
    print(f"Database file {db_filename} has been created.")

# using function
db_filename = 'trend_analysis_contributions.db'
drop_database(db_filename)  # Delete database
create_database(db_filename)  # Create database

Database file trend_analysis_contributions.db has been deleted.
Database file trend_analysis_contributions.db has been created.


# Insert Data

In [97]:
import sqlite3
import pandas as pd

def insert_dates_to_table(db_filename, df1):
    # Connecting to a SQLite Database
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()

    # Prepare SQL statements to insert data
    insert_date = """
    INSERT INTO Dates (date) VALUES (?)
    """

    # Make sure that the data in the Date column is of string type and handle null values
    df1['Date'] = df1['Date'].astype(str)
    df1 = df1.dropna(subset=['Date'])  # delete null value

    # Traverse the Date column of df1 and insert it into the database
    for index, row in df1.iterrows():
        # Perform an insert operation
        cursor.execute(insert_date, (row['Date'],))

    # confirm opearation
    conn.commit()
    print("Dates have been inserted into the Dates table.")

    # close connection
    conn.close()

# define database
db_filename = 'trend_analysis_contributions.db'

# using function
insert_dates_to_table(db_filename, df1)

Dates have been inserted into the Dates table.


In [98]:
import sqlite3
import pandas as pd

def insert_factors_to_table(db_filename, df1):
    # Connecting to a SQLite Database
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()

    # Prepare SQL statements to insert data
    insert_factor = """
    INSERT INTO Factors (factor_name) VALUES (?)
    """

    # Traverse the column names of df1 starting from the third column and insert them into the database
    for column_name in df1.columns[2:]:  # Starting from the third column, index is 2
        # Perform an insert operation
        cursor.execute(insert_factor, (column_name,))

    # confirm operation
    conn.commit()
    print("Factors have been inserted into the Factors table.")

    # close connection
    conn.close()

# define database
db_filename = 'trend_analysis_contributions.db'

# using function
insert_factors_to_table(db_filename, df1)

Factors have been inserted into the Factors table.


In [99]:
import sqlite3
import pandas as pd

def insert_contributions_to_table(db_filename, df1):
    try:
        # Connecting to a SQLite Database
        conn = sqlite3.connect(db_filename)
        cursor = conn.cursor()
        print("Connected to the database.")

        # Prepare SQL statement to query date_id
        get_date_id = "SELECT date_id FROM Dates WHERE date = ?"

        # Prepare the SQL statement to query factor_id
        get_factor_id = "SELECT factor_id FROM Factors WHERE factor_name = ?"

        # Prepare SQL statements to insert into the Contributions table
        insert_contribution = """
        INSERT INTO Contributions (date_id, factor_id, contribution_value) VALUES (?, ?, ?)
        """

        # Traverse each row of df1
        for index, row in df1.iterrows():
            # Get date_id
            cursor.execute(get_date_id, (str(row['Date']),))
            date_id_result = cursor.fetchone()
            if date_id_result:
                date_id = date_id_result[0]
                print(f"Found date_id {date_id} for date {row['Date']}")
            else:
                print(f"No matching date_id found for date {row['Date']}")
                continue

            # Iterate through the column names and values ​​of df1 starting from the third column
            for column_name in df1.columns[2:]:
                # 获取factor_id
                cursor.execute(get_factor_id, (str(column_name),))
                factor_id_result = cursor.fetchone()
                if factor_id_result:
                    factor_id = factor_id_result[0]
                    print(f"Found factor_id {factor_id} for factor {column_name}")
                    # insert contribution value
                    cursor.execute(insert_contribution, (date_id, factor_id, row[column_name]))
                    print(f"Inserted contribution value {row[column_name]} for date_id {date_id} and factor_id {factor_id}")
                else:
                    print(f"No matching factor_id found for factor {column_name}")

        # confirm opeartion
        conn.commit()
        print("Contributions have been inserted into the Contributions table.")

        # close connection
        conn.close()
        print("Database connection closed.")
    except Exception as e:
        print(f"An error occurred: {e}")

# define database
db_filename = 'trend_analysis_contributions.db'

# using function
insert_contributions_to_table(db_filename, df1)



Streaming output truncated to the last 5000 lines.
Inserted contribution value 0.56058 for date_id 3343 and factor_id 6
Found factor_id 7 for factor Sulphate/Marine diesel
Inserted contribution value 0.24704 for date_id 3343 and factor_id 7
Found date_id 3344 for date 2016-12-25
Found factor_id 1 for factor Soil/Road dust
Inserted contribution value 0.29058 for date_id 3344 and factor_id 1
Found factor_id 2 for factor Sea salt
Inserted contribution value 1.2739 for date_id 3344 and factor_id 2
Found factor_id 3 for factor Diesel vehicles
Inserted contribution value 0.31228 for date_id 3344 and factor_id 3
Found factor_id 4 for factor Biomass burning
Inserted contribution value 0.91906 for date_id 3344 and factor_id 4
Found factor_id 5 for factor Construction
Inserted contribution value 0.065909 for date_id 3344 and factor_id 5
Found factor_id 6 for factor Petrol vehicles
Inserted contribution value 0.61647 for date_id 3344 and factor_id 6
Found factor_id 7 for factor Sulphate/Marine di

# Testing

In [100]:
# 验证factor table
def verify_factors_in_database(db_filename, df1):
    # 连接到SQLite数据库
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()
    print("Connected to the database.")

    # 查询Factors表中的所有数据
    try:
        cursor.execute("SELECT * FROM Factors")
        factors_from_db = cursor.fetchall()
        print("Data fetched from the Factors table.")
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return

    # 将查询结果转换为DataFrame以便比较
    factors_df = pd.DataFrame(factors_from_db, columns=['factor_id', 'factor_name'])
    print("Converted fetched data to DataFrame.")

    # 获取df1的第三列开始的列名
    expected_factors = df1.columns[2:].tolist()
    print("Expected factors:", expected_factors)

    # 检查数据库中的因子是否与df1中的因子相匹配
    if set(factors_df['factor_name']) == set(expected_factors):
        print("Verification successful: All factors from df1 are present in the Factors table.")
    else:
        print("Verification failed: There are discrepancies between df1 and the Factors table.")

    # 关闭连接
    conn.close()
    print("Database connection closed.")

# 使用验证函数
verify_factors_in_database(db_filename, df1)

Connected to the database.
Data fetched from the Factors table.
Converted fetched data to DataFrame.
Expected factors: ['Soil/Road dust', 'Sea salt', 'Diesel vehicles', 'Biomass burning', 'Construction', 'Petrol vehicles', 'Sulphate/Marine diesel']
Verification successful: All factors from df1 are present in the Factors table.
Database connection closed.


In [101]:
import sqlite3
from datetime import datetime

def query_contribution(db_filename, date_str, factor_name):
    # 连接到SQLite数据库
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()

    # 将日期字符串转换为 YYYY-MM-DD 格式
    try:
        # 尝试解析日期字符串，假设格式为 'M/D/YYYY'
        parsed_date = datetime.strptime(date_str, '%m/%d/%Y')
        # 格式化为 'YYYY-MM-DD'
        date_str_formatted = parsed_date.strftime('%Y-%m-%d')
    except ValueError:
        print("Date string is not in the expected format 'M/D/YYYY'")
        return

    # 准备查询date_id的SQL语句
    get_date_id = "SELECT date_id FROM Dates WHERE date = ?"

    # 准备查询factor_id的SQL语句
    get_factor_id = "SELECT factor_id FROM Factors WHERE factor_name = ?"

    # 准备查询贡献值的SQL语句
    get_contribution = """
    SELECT contribution_value FROM Contributions
    WHERE date_id = (SELECT date_id FROM Dates WHERE date = ?)
    AND factor_id = (SELECT factor_id FROM Factors WHERE factor_name = ?)
    """

    try:
        # 获取date_id
        cursor.execute(get_date_id, (date_str_formatted,))
        date_id_result = cursor.fetchone()
        if date_id_result:
            date_id = date_id_result[0]
            print(f"Found date_id {date_id} for date {date_str_formatted}")
        else:
            print(f"No matching date_id found for date {date_str_formatted}")
            return

        # 获取factor_id
        cursor.execute(get_factor_id, (factor_name,))
        factor_id_result = cursor.fetchone()
        if factor_id_result:
            factor_id = factor_id_result[0]
            print(f"Found factor_id {factor_id} for factor {factor_name}")
            # 查询贡献值
            cursor.execute(get_contribution, (date_str_formatted, factor_name,))
            contribution_result = cursor.fetchone()
            if contribution_result:
                print(f"Contribution value for {date_str_formatted} and {factor_name} is {contribution_result[0]}")
            else:
                print(f"No contribution value found for {date_str_formatted} and {factor_name}")
        else:
            print(f"No matching factor_id found for factor {factor_name}")
    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # 关闭连接
        conn.close()

# 使用函数查询数据
db_filename = 'trend_analysis_contributions.db'
query_date = '1/30/2006'
query_factor = 'Soil/Road dust'
query_contribution(db_filename, query_date, query_factor)

Found date_id 22 for date 2006-01-30
Found factor_id 1 for factor Soil/Road dust
Contribution value for 2006-01-30 and Soil/Road dust is 0.24937
